In [2]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy import ndimage
from skimage import filters, morphology, io
import shutil
import warnings

In [4]:
# Define the mapping of specific values to RGB colors
value_to_rgb = {
    1: [255, 48, 48],      # fire brick1
    2: [64, 224, 208],     # turquoise
    3: [72, 118, 255],     # royalblue1
    4: [255, 255, 0],      # yellow
    5: [255, 255, 255],    # white
    6: (192, 192, 192),    # silver
    7: [255, 0, 255],      # magenta
}

In [ ]:
file_path = '/data/nas/Cth/MRI_segmentation/'
file_list = os.listdir(file_path)
print(file_list)

In [6]:
# Loading real path / if you want to crop other files, you should write its folder's path here!
def tar_space(png):
    file_path = '/data/nas/Cth/MRI_segmentation/'
    file_path_01 = os.path.join(file_path,png,f"{png}_nuc_t1_resample_axial2d")
    file_path_02 = os.path.join(file_path,png,"GT_rCSF_apLVc5_hippCSF")
    file_path_03 = os.path.join(file_path,png,"GT_Hipp_LatVentc5_Cortex")
    return file_path_01,file_path_02,file_path_03

In [7]:
# Control ostu area
def create_brain_mask(image, adjustment_factor=0.2):
    threshold = filters.threshold_otsu(image)
    adjusted_threshold = threshold * adjustment_factor
    binary_mask = image > adjusted_threshold
    labeled_array, num_features = ndimage.label(binary_mask)
    sizes = ndimage.sum(binary_mask, labeled_array, range(num_features + 1))
    mask = sizes == sizes.max()
    brain_mask = mask[labeled_array]
    convex_hull = morphology.convex_hull_image(brain_mask)

    # Make rect mask
    rows = np.any(convex_hull, axis=1)
    cols = np.any(convex_hull, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    rectangular_mask = np.zeros_like(image, dtype=bool)
    rectangular_mask[min_row:max_row+1, min_col:max_col+1] = True
    
    return rectangular_mask
    

In [8]:
# Crop mask
def crop_mask(image, mask):
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    cropped_image = image[min_row:max_row+1, min_col:max_col+1]
    
    return cropped_image

In [9]:
# Make new .nii file
def make_nii(or_nii_data, new_nii_data, save_path, file_name):
    output_nii_file = os.path.join(save_path, f"{file_name}.nii")
    extracted_nii = nib.Nifti1Image(new_nii_data, or_nii_data.affine)
    nib.save(extracted_nii, output_nii_file)

In [10]:
# Padding
def pading(crop):
    sq_size = 0
    if crop.shape[0] > crop.shape[1]:
        sq_size = crop.shape[0]
        pad_r = int((sq_size - crop.shape[0]) / 2)
        pad_c = int((sq_size - crop.shape[1]) / 2)
        if ((sq_size - crop.shape[1]) % 2) == 1:
            pad = np.pad(crop, ((pad_r, pad_r), (pad_c, pad_c + 1)), 'constant', constant_values=0)
        else:
            pad = np.pad(crop, ((pad_r, pad_r), (pad_c, pad_c)), 'constant', constant_values=0)

    else:
        sq_size = crop.shape[1]
        pad_r = int((sq_size - crop.shape[0]) / 2)
        pad_c = int((sq_size - crop.shape[1]) / 2)

        if ((sq_size - crop.shape[1]) % 2) == 1:
            pad = np.pad(crop, ((pad_r, pad_r + 1), (pad_c, pad_c)), 'constant', constant_values=0)
        else:
            pad = np.pad(crop, ((pad_r, pad_r), (pad_c, pad_c)), 'constant', constant_values=0)

    return pad

In [11]:
# Map values to RGB colors
def map_values_to_rgb(img, value_to_rgb):
    # Create an empty array for the RGB image
    rgb_img = np.zeros((*img.shape, 3), dtype=np.uint8)

    for value, color in value_to_rgb.items():
        mask = (img == value)
        rgb_img[mask] = color

    return rgb_img

In [12]:
# Make new .jpeg file - Colorful
def make_jpeg(new_nii_data, save_path, file_name):
    # Map values to RGB
    rgb_img = map_values_to_rgb(new_nii_data, value_to_rgb)
    
    # Save the RGB image as a png file
    output_png_file = os.path.join(save_path, f"{file_name}.jpeg")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        io.imsave(output_png_file, rgb_img)

In [13]:
# Make new .jpeg file - gray
def save_jpeg(new_nii_data, save_path, file_name):
    fig, ax = plt.subplots(figsize=(new_nii_data.shape[1] / 100, new_nii_data.shape[0] / 100), dpi=100)
    ax.imshow(new_nii_data, cmap='gray')
    ax.axis('off')
    output_png_file = os.path.join(save_path, f"{file_name}.jpeg")
    plt.savefig(output_png_file, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

In [ ]:
# Main code
for i in file_list:
    print(i) # Patient ID
    nii_path = list(tar_space(i))
    num = 0

    # Find biggest mask
    mask = None
    mask_area = 0

    for j in nii_path:
        print(j) # File path
        fin_file_path = os.path.join(f'{j}sq')
        # Delete garbage file
        #if os.path.exists(fin_file_path) and os.path.isdir(fin_file_path):
        #    shutil.rmtree(fin_file_path)
        # Making clear folder
        os.makedirs(fin_file_path, exist_ok=True)
        
        total_files = os.listdir(j)

        # Making & Choosing mask
        for k in total_files:
            if num == 0:
                k_path = os.path.join(j,k)
                nii_data_1 = nib.load(k_path)
                nii_data_1_nii = nii_data_1.get_fdata()
                nii_data_1_np = np.rot90(nii_data_1_nii)
                mask1 = create_brain_mask(nii_data_1_np)
                area = np.sum(mask1)
                if area > mask_area:
                    mask = mask1
                    mask_area = area

        # Making final files
        for k in total_files:
            # Not GT file -> no making jpeg files
            if num == 0:
                kp = k.split('.')[0]
                k_path = os.path.join(j,k)
                nii_data_1 = nib.load(k_path)
                nii_data_1_nii = nii_data_1.get_fdata()
                nii_data_1_np = np.rot90(nii_data_1_nii)
                crop = crop_mask(nii_data_1_np, mask)
                pad = pading(crop)
                make_nii(nii_data_1,np.rot90(pad,3),fin_file_path,kp)

            # GT file -> making jpeg files
            else:
                if k.endswith('.nii'):
                    kp = k.split('.')[0]
                    k_path = os.path.join(j,k)
                    nii_data_1 = nib.load(k_path)
                    nii_data_1_nii = nii_data_1.get_fdata()
                    nii_data_1_np = np.rot90(nii_data_1_nii)
                    crop = crop_mask(nii_data_1_np, mask)
                    pad = pading(crop)
                    make_nii(nii_data_1,np.rot90(pad,3),fin_file_path,kp)
                    make_jpeg(pad,fin_file_path,kp)

        num+=1

In [ ]:
# Main code
for i in file_list[836:]:
    print(i) # Patient ID
    nii_path = list(tar_space(i))
    num = 0

    # Find biggest mask
    mask = None
    mask_area = 0

    for j in nii_path:
        print(j) # File path
        fin_file_path = os.path.join(f'{j}sq')
        # Delete garbage file
        #if os.path.exists(fin_file_path) and os.path.isdir(fin_file_path):
        #    shutil.rmtree(fin_file_path)
        # Making clear folder
        os.makedirs(fin_file_path, exist_ok=True)
        
        total_files = os.listdir(j)

        # Making & Choosing mask
        for k in total_files:
            if num == 0:
                k_path = os.path.join(j,k)
                nii_data_1 = nib.load(k_path)
                nii_data_1_nii = nii_data_1.get_fdata()
                nii_data_1_np = np.rot90(nii_data_1_nii)
                mask1 = create_brain_mask(nii_data_1_np)
                area = np.sum(mask1)
                if area > mask_area:
                    mask = mask1
                    mask_area = area

        # Making final files
        for k in total_files:
            # Not GT file -> no making jpeg files
            if num == 0:
                kp = k.split('.')[0]
                k_path = os.path.join(j,k)
                nii_data_1 = nib.load(k_path)
                nii_data_1_nii = nii_data_1.get_fdata()
                nii_data_1_np = np.rot90(nii_data_1_nii)
                crop = crop_mask(nii_data_1_np, mask)
                pad = pading(crop)
                make_nii(nii_data_1,np.rot90(pad,3),fin_file_path,kp)

            # GT file -> making jpeg files
            else:
                if k.endswith('.nii'):
                    kp = k.split('.')[0]
                    k_path = os.path.join(j,k)
                    nii_data_1 = nib.load(k_path)
                    nii_data_1_nii = nii_data_1.get_fdata()
                    nii_data_1_np = np.rot90(nii_data_1_nii)
                    crop = crop_mask(nii_data_1_np, mask)
                    pad = pading(crop)
                    make_nii(nii_data_1,np.rot90(pad,3),fin_file_path,kp)
                    make_jpeg(pad,fin_file_path,kp)

        num+=1